In [2]:
import pandas as pd
import re
import json
import csv
from datetime import datetime
import logging

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
orders_json = [
    {
        "Order": "2ABD90DB",
        "Delivered": "The order was delivered at 11:57 PM on September 26, 2023.",
        "Pick Up Location": "1402 Williams Dr, Georgetown, TX 78628, USA",
        "Order Details": {
            "1 \u00d7 19 Crimes Cabernet Sauvignon 750ml Bottle (14% ABV) (Specialty Wine)": "$13.79"
        },
        "Subtotal": "$13.79",
        "Subtotal Tax": "$1.76",
        "Commission (16%)": "-$2.21",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$13.34",
        "Transaction #8072672500 - Delivery": "$13.34"
    },
    {
        "Order": "42CEA1A6",
        "Delivered": "The order was delivered at 11:25 PM on September 26, 2023.",
        "Pick Up Location": "1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA",
        "Order Details": {
            "1 \u00d7 Ice Breakers Mint Coolmint (1.5 oz) (Mint TX)": "$5.69"
        },
        "Subtotal": "$5.69",
        "Subtotal Tax": "$0.73",
        "Commission (20%)": "-$1.14",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$5.28",
        "Transaction #8072574397 - Delivery": "$5.28"
    },
    {
        "Order": "6AB38CB1",
        "Delivered": "The order was delivered at 11:24 PM on September 26, 2023.",
        "Pick Up Location": "1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA",
        "Order Details": {
            "2 \u00d7 Ice Breakers Mint Coolmint (1.5 oz) (Mint TX)": "$11.38"
        },
        "Subtotal": "$11.38",
        "Subtotal Tax": "$1.45",
        "Commission (20%)": "-$2.28",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$10.55",
        "Transaction #8072573625 - Delivery": "$10.55"
    },
    {
        "Order": "4379B60E",
        "Delivered": "The order was delivered at 10:52 PM on September 26, 2023.",
        "Pick Up Location": "7045 W Del Mar Blvd, Laredo, TX 78041, USA",
        "Order Details": {
            "1 \u00d7 Smirnoff Ice Original Malt Beverage Bottle (11.2 oz x 6 ct) (Flavored Malt Beverages Laredo)": "$14.99"
        },
        "Subtotal": "$14.99",
        "Subtotal Tax": "$1.91",
        "Commission (16%)": "-$2.40",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$14.50",
        "Transaction #8072378844 - Delivery": "$14.50"
    },
    {
        "Order": "0A150B93",
        "Delivered": "The order was delivered at 10:34 PM on September 26, 2023.",
        "Pick Up Location": "2503 TX-349, Midland, TX 79706, USA",
        "Order Details": {
            "1 \u00d7 Cheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)": "$3.19"
        },
        "Subtotal": "$3.19",
        "Subtotal Tax": "$0.00",
        "Commission (16%)": "-$0.51",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$2.68",
        "Transaction #8072265117 - Delivery": "$2.68"
    },
    {
        "Order": "E0EB8837",
        "Delivered": "The order was delivered at 8:18 PM on September 26, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "3 \u00d7 Steel Reserve Original Can (25 oz) (Flavored Malt Beverages NT)": "$13.17"
        },
        "Subtotal": "$13.17",
        "Subtotal Tax": "$1.68",
        "Commission (16%)": "-$2.11",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$12.74",
        "Transaction #8071200562 - Delivery": "$12.74"
    },
    {
        "Order": "77896463",
        "Customer Picked Up": "The customer picked up the order at 8:05 PM on September 26, 2023.",
        "Pick Up Location": "7045 W Del Mar Blvd, Laredo, TX 78041, USA",
        "Order Details": {
            "1 \u00d7 Natural Light Beer Can (12 oz x 15 ct) (Domestic Beer Laredo)": "$13.99"
        },
        "Subtotal": "$13.99",
        "Subtotal Tax": "$1.78",
        "Commission (9%)": "-$1.26",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$14.51"
    },
    {
        "Order": "B48F2460",
        "Delivered": "The order was delivered at 7:46 PM on September 26, 2023.",
        "Pick Up Location": "14620 Ranch Rd 12, Wimberley, TX 78676, USA",
        "Order Details": {
            "3 \u00d7 Cayman Jack Margarita Can (19 oz) (Flavored Malt Beverages CT)": "$17.07"
        },
        "Subtotal": "$17.07",
        "Subtotal Tax": "$2.18",
        "Commission (23%)": "-$3.93",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$15.32",
        "Transaction #8070880369 - Delivery": "$15.32"
    },
    {
        "Order": "78588F69",
        "Delivered": "The order was delivered at 6:35 PM on September 26, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "2 \u00d7 Ice Large (Fountain, Frozen, & Tea)": "$14.98"
        },
        "Subtotal": "$14.98",
        "Subtotal Tax": "$1.91",
        "Commission (16%)": "-$2.40",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$14.49",
        "Transaction #8070265277 - Delivery": "$14.49"
    },
    {
        "Order": "5C857863",
        "Cancelled - Not Paid": "The order was cancelled on September 26, 2023 at 5:59 PM. You were not paid because you did not confirm the order before the order was cancelled.",
        "Pick Up Location": "14620 Ranch Rd 12, Wimberley, TX 78676, USA",
        "Order Details": {
            "3 \u00d7 Cayman Jack Margarita Can (19 oz) (Flavored Malt Beverages CT)": "$17.07"
        },
        "Subtotal": "$17.07",
        "Subtotal Tax": "$2.18",
        "Commission (23%)": "-$3.93",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$0.00"
    },
    {
        "Order": "CA2588A4",
        "Delivered": "The order was delivered at 4:02 PM on September 26, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "3 \u00d7 Steel Reserve Original Can (25 oz) (Flavored Malt Beverages NT)": "$13.17"
        },
        "Subtotal": "$13.17",
        "Subtotal Tax": "$1.68",
        "Commission (16%)": "-$2.11",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$12.74",
        "Transaction #8068891159 - Delivery": "$12.74"
    },
    {
        "Order": "B6233BDE",
        "Delivered": "The order was delivered at 11:55 AM on September 26, 2023.",
        "Pick Up Location": "1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA",
        "Order Details": {
            "2 \u00d7 Gatorade Lime Cucumber Bottle (28 oz) (Sports Drinks TX)": "$9.58",
            " 1 \u00d7 Arizona Green Tea Bottle (34oz) (Tea TX)": "$4.39",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Coke": "$0.00",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Mountain Dew": "$0.00",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Gatorade Lemon-Lime": "$0.00"
        },
        "Subtotal": "$13.97",
        "Subtotal Tax": "$1.22",
        "Commission (23%)": "-$3.21",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$11.98",
        "Transaction #8067195196 - Delivery": "$11.98"
    },
    {
        "Order": "8903B15F",
        "Delivered": "The order was delivered at 11:41 AM on September 26, 2023.",
        "Pick Up Location": "800 W Interstate 20, Big Spring, TX 79720, USA",
        "Order Details": {
            "1 \u00d7 Fritos Scoops Corn Chips (9.25 oz) (Chips & Popcorn TX)": "$6.49",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Diet Dr Pepper": "$0.00",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Coke": "$0.00"
        },
        "Subtotal": "$6.49",
        "Subtotal Tax": "$0.00",
        "Commission (16%)": "-$1.04",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$5.45",
        "Transaction #8067145629 - Delivery": "$5.45"
    },
    {
        "Order": "9EAC6D9E",
        "Cancelled - Not Paid": "The order was cancelled on September 26, 2023 at 10:39 AM. You were not paid because you did not confirm the order before the order was cancelled.",
        "Pick Up Location": "3071 University Dr E, Bryan, TX 77802, USA",
        "Order Details": {
            "2 \u00d7 Bacon Egg Cheese Biscuit (Breakfast Sandwiches & Bowls)": "$9.78",
            " 1 \u00d7 Monster Java Loca Mocha Can (15oz) (Coffee TX)": "$5.69",
            " 3 \u00d7 Potato Egg Cheese Taco (Breakfast Tacos & Quesadillas) Salsa: Salsa Verde": "$14.07"
        },
        "Subtotal": "$29.54",
        "Subtotal Tax": "$3.77",
        "Commission (23%)": "-$6.79",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$0.00"
    },
    {
        "Order": "ABDD1702",
        "Delivered": "The order was delivered at 12:00 AM on September 26, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "1 \u00d7 Coors Original Beer Bottle (12oz x 6ct) (Domestic Beer NT)": "$14.99"
        },
        "Subtotal": "$14.99",
        "Subtotal Tax": "$1.91",
        "Commission (23%)": "-$3.45",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$13.45",
        "Transaction #8065637009 - Delivery": "$13.45"
    }
]

In [4]:
master_dataset_file_path = '/Users/ekim/workspace/personal/dd-bot/dev/store_list.xlsx'


In [69]:
def read_masterdataset_excel():
    master_df = pd.read_excel(master_dataset_file_path, index_col=0)
    return master_df

master_df = read_masterdataset_excel()


def add_cols_to_masterset_df(master_df):

    # Add street_num col to df
    master_df = pd.concat([master_df, street_num_df], axis=1)

    # Cast int64 default dtype cols to strings for proper JOINs
    master_df['Site #'] = master_df['Site #'].astype(str)
    master_df['Zip'] = master_df['Zip'].astype(str)
    return master_df

master_df = add_cols_to_masterset_df(master_df)
master_df

,Site #,Site Description,Address,City,County,State,Zip,Notes,street_num
Store #,,,,,,,,,
1,1,Kwik Chek #1,2226 North Center,Bonham,Fannin,TX,75418,NaN,2226
3,3,Kwik Chek #3,971 FM 273,Bonham,Fannin,TX,75418,NaN,971
5,5,Kwik Chek #5,NaN,Bonham,Fannin,TX,75418,Not Open Yet,nan
6,6,Kwik Chek #6,816 North Center,Bonham,Fannin,TX,75418,NaN,816
7,7,Kwik Chek #7,101 Hwy 69,Celeste,Hunt,TX,75423,NaN,101
8,8,Kwik Chek #8,308 Hwy 69,Leonard,Fannin,TX,75452,NaN,308
10,10,Kwik Chek #10,1741 North Hwy 121,Bonham,Fannin,TX,75418,NaN,1741
12,12,Kwik Chek #12,102 S Hwy 78,Achille,Bryan,OK,74720,NaN,102
13,13,Kwik Chek #13,102 West Main,Calera,Bryan,OK,74730,NaN,102


In [ ]:
# todo: logic to clean up data for instances of "        "Order": "No Delivery has been found."

def remove_incomplete_orders(orders):
    complete_orders = []
    for order in orders:
        print(order)
        if len(order) >= 7:
            complete_orders.append(order)
    return complete_orders

orders = remove_incomplete_orders(orders_json)
orders


In [65]:
def order_id_to_pickup_location(orders):
    order_id_to_pickup_location  = {}
    for order in orders:
        order_id = order['Order']
        store_addrs = order['Pick Up Location']
        order_id_to_pickup_location[order_id] = store_addrs
    return order_id_to_pickup_location
order_to_location = order_id_to_pickup_location(orders)
order_to_location

{'2ABD90DB': '1402 Williams Dr, Georgetown, TX 78628, USA',
 '42CEA1A6': '1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA',
 '6AB38CB1': '1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA',
 '4379B60E': '7045 W Del Mar Blvd, Laredo, TX 78041, USA',
 '0A150B93': '2503 TX-349, Midland, TX 79706, USA',
 'E0EB8837': '5004 Wesley St, Greenville, TX 75402, USA',
 '77896463': '7045 W Del Mar Blvd, Laredo, TX 78041, USA',
 'B48F2460': '14620 Ranch Rd 12, Wimberley, TX 78676, USA',
 '78588F69': '5004 Wesley St, Greenville, TX 75402, USA',
 '5C857863': '14620 Ranch Rd 12, Wimberley, TX 78676, USA',
 'CA2588A4': '5004 Wesley St, Greenville, TX 75402, USA',
 'B6233BDE': '1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA',
 '8903B15F': '800 W Interstate 20, Big Spring, TX 79720, USA',
 '9EAC6D9E': '3071 University Dr E, Bryan, TX 77802, USA',
 'ABDD1702': '5004 Wesley St, Greenville, TX 75402, USA'}

In [66]:
def get_raw_order_to_location_df():
    raw_orders_to_location = order_id_to_pickup_location(orders)
    order_to_location_pairs = list(raw_orders_to_location.items())
    order_to_location_df = pd.DataFrame(order_to_location_pairs, columns=['order_id', 'pickup_location'])
    
    return order_to_location_df
___get_raw_order_to_location_df = get_raw_order_to_location_df()
___get_raw_order_to_location_df

,order_id,pickup_location
0,2ABD90DB,"1402 Williams Dr, Georgetown, TX 78628, USA"
1,42CEA1A6,"1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA"
2,6AB38CB1,"1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA"
3,4379B60E,"7045 W Del Mar Blvd, Laredo, TX 78041, USA"
4,0A150B93,"2503 TX-349, Midland, TX 79706, USA"
5,E0EB8837,"5004 Wesley St, Greenville, TX 75402, USA"
6,77896463,"7045 W Del Mar Blvd, Laredo, TX 78041, USA"
7,B48F2460,"14620 Ranch Rd 12, Wimberley, TX 78676, USA"
8,78588F69,"5004 Wesley St, Greenville, TX 75402, USA"
9,5C857863,"14620 Ranch Rd 12, Wimberley, TX 78676, USA"


In [35]:
master_df

,Site #,Site Description,Address,City,County,State,Zip,Notes,street_num
Store #,,,,,,,,,
1,1,Kwik Chek #1,2226 North Center,Bonham,Fannin,TX,75418,NaN,2226
3,3,Kwik Chek #3,971 FM 273,Bonham,Fannin,TX,75418,NaN,971
5,5,Kwik Chek #5,NaN,Bonham,Fannin,TX,75418,Not Open Yet,nan
6,6,Kwik Chek #6,816 North Center,Bonham,Fannin,TX,75418,NaN,816
7,7,Kwik Chek #7,101 Hwy 69,Celeste,Hunt,TX,75423,NaN,101
8,8,Kwik Chek #8,308 Hwy 69,Leonard,Fannin,TX,75452,NaN,308
10,10,Kwik Chek #10,1741 North Hwy 121,Bonham,Fannin,TX,75418,NaN,1741
12,12,Kwik Chek #12,102 S Hwy 78,Achille,Bryan,OK,74720,NaN,102
13,13,Kwik Chek #13,102 West Main,Calera,Bryan,OK,74730,NaN,102


In [53]:
___get_raw_order_to_location_df['pickup_location'].str.replace(', USA', '')  # remove usa with preceeding comma

0             1402 Williams Dr, Georgetown, TX 78628
1        1301 N U.S. Hwy 281, Marble Falls, TX 78654
2        1301 N U.S. Hwy 281, Marble Falls, TX 78654
3              7045 W Del Mar Blvd, Laredo, TX 78041
4                     2503 TX-349, Midland, TX 79706
5               5004 Wesley St, Greenville, TX 75402
6              7045 W Del Mar Blvd, Laredo, TX 78041
7             14620 Ranch Rd 12, Wimberley, TX 78676
8               5004 Wesley St, Greenville, TX 75402
9             14620 Ranch Rd 12, Wimberley, TX 78676
10              5004 Wesley St, Greenville, TX 75402
11    1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495
12         800 W Interstate 20, Big Spring, TX 79720
13             3071 University Dr E, Bryan, TX 77802
14              5004 Wesley St, Greenville, TX 75402
Name: pickup_location, dtype: object

In [56]:

zip_code_pattern = r'(?P<zip_code>\d{5})'
zip_code_df = ___get_raw_order_to_location_df['pickup_location'].str.extract(zip_code_pattern)

state_pattern = r'(?P<state>[A-Z]{2})'
state_df = ___get_raw_order_to_location_df['pickup_location'].str.extract(state_pattern)

street_num_pattern = r'(?P<street_num>^.+?)\s'
street_num_df = ___get_raw_order_to_location_df['pickup_location'].str.extract(street_num_pattern)


new_df = pd.concat([street_num_df, state_df, zip_code_df], axis=1)
new_df

,street_num,state,zip_code
0,1402,TX,78628
1,1301,TX,78654
2,1301,TX,78654
3,7045,TX,78041
4,2503,TX,79706
5,5004,TX,75402
6,7045,TX,78041
7,14620,TX,14620
8,5004,TX,75402
9,14620,TX,14620


In [59]:
# todo: use zip code from above to merge with master_df[Zip] 

new_merged_df = pd.merge(master_df, new_df, left_on=['Zip', 'State', 'street_num'], right_on=['zip_code', 'state', 'street_num' ], how='inner')
new_merged_df


,Site #,Site Description,Address,City,County,State,Zip,Notes,street_num,state,zip_code
0,20,Kwik Chek #20,5004 Wesley Street,Greenville,Hunt,TX,75402,NaN,5004,TX,75402
1,20,Kwik Chek #20,5004 Wesley Street,Greenville,Hunt,TX,75402,NaN,5004,TX,75402
2,20,Kwik Chek #20,5004 Wesley Street,Greenville,Hunt,TX,75402,NaN,5004,TX,75402
3,20,Kwik Chek #20,5004 Wesley Street,Greenville,Hunt,TX,75402,NaN,5004,TX,75402
4,32,TXB 32,1001 Van Alstyne Parkway,Van Alstyne,Grayson,TX,75495,NaN,1001,TX,75495
5,53,Kwik Chek #53,1301 Hwy 281 North,Marble Falls,Burnet,TX,78654,NaN,1301,TX,78654
6,53,Kwik Chek #53,1301 Hwy 281 North,Marble Falls,Burnet,TX,78654,NaN,1301,TX,78654
7,65,Kwik Chek #65,1402 Williams Drive,Georgetown,Williamson,TX,78628,NaN,1402,TX,78628
8,70,Kwik Chek #70,800 IH 20 West,Big Spring,Howard,TX,79720,NaN,800,TX,79720
9,71,TXB 71,2503 Rankin Hwy,Midland,Midland,TX,79706,NaN,2503,TX,79706


In [37]:
def splitup_pickup_location():
    order_to_location_df['pickup_location'] = ___get_raw_order_to_location_df['pickup_location'].str.replace(',', '')
    
    # Remove "USA" substring
    order_to_location_df['pickup_location'] = order_to_location_df['pickup_location'].str.replace('USA', '')

    # 
    pattern = r'(?P<address>^.+?)\s(?P<city>\w+\s*\w*)\s(?P<state>[A-Z]{2})\s(?P<zip_code>\d{5})'
    # 
    df_extracted = order_to_location_df['pickup_location'].str.extract(pattern)
    # 
    _order_to_location_df = pd.concat([order_to_location_df, df_extracted], axis=1)

    return _order_to_location_df

_order_to_location_df = splitup_pickup_location()
_order_to_location_df

,order_id,pickup_location,address,city,state,zip_code
0,2ABD90DB,1402 Williams Dr Georgetown TX 78628,1402 Williams,Dr Georgetown,TX,78628
1,42CEA1A6,1301 N U.S. Hwy 281 Marble Falls TX 78654,1301 N U.S. Hwy 281,Marble Falls,TX,78654
2,6AB38CB1,1301 N U.S. Hwy 281 Marble Falls TX 78654,1301 N U.S. Hwy 281,Marble Falls,TX,78654
3,4379B60E,7045 W Del Mar Blvd Laredo TX 78041,7045 W Del Mar,Blvd Laredo,TX,78041
4,0A150B93,2503 TX-349 Midland TX 79706,2503 TX-349,Midland,TX,79706
5,E0EB8837,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402
6,77896463,7045 W Del Mar Blvd Laredo TX 78041,7045 W Del Mar,Blvd Laredo,TX,78041
7,B48F2460,14620 Ranch Rd 12 Wimberley TX 78676,14620 Ranch Rd,12 Wimberley,TX,78676
8,78588F69,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402
9,5C857863,14620 Ranch Rd 12 Wimberley TX 78676,14620 Ranch Rd,12 Wimberley,TX,78676


In [19]:
def add_street_num_col():
    street_num_pattern = r'(?P<street_num>\b\d{3,5}\b)'

    addrs_extracted_df = _order_to_location_df['address'].str.extract(street_num_pattern)

    ADD_order_to_location_df = pd.concat([_order_to_location_df, addrs_extracted_df], axis=1)
    return ADD_order_to_location_df

ADD_order_to_location_df = add_street_num_col()
ADD_order_to_location_df

,order_id,pickup_location,address,city,state,zip_code,street_num
0,2ABD90DB,1402 Williams Dr Georgetown TX 78628,1402 Williams,Dr Georgetown,TX,78628,1402
1,42CEA1A6,1301 N U.S. Hwy 281 Marble Falls TX 78654,1301 N U.S. Hwy 281,Marble Falls,TX,78654,1301
2,6AB38CB1,1301 N U.S. Hwy 281 Marble Falls TX 78654,1301 N U.S. Hwy 281,Marble Falls,TX,78654,1301
3,4379B60E,7045 W Del Mar Blvd Laredo TX 78041,7045 W Del Mar,Blvd Laredo,TX,78041,7045
4,0A150B93,2503 TX-349 Midland TX 79706,2503 TX-349,Midland,TX,79706,2503
5,E0EB8837,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402,5004
6,77896463,7045 W Del Mar Blvd Laredo TX 78041,7045 W Del Mar,Blvd Laredo,TX,78041,7045
7,B48F2460,14620 Ranch Rd 12 Wimberley TX 78676,14620 Ranch Rd,12 Wimberley,TX,78676,14620
8,78588F69,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402,5004
9,5C857863,14620 Ranch Rd 12 Wimberley TX 78676,14620 Ranch Rd,12 Wimberley,TX,78676,14620


In [23]:
def get_merged_and_organized_master_df():
    merged_df = pd.merge(master_df, ADD_order_to_location_df, left_on=['Zip', 'street_num'], right_on=['zip_code', 'street_num'], how='inner')

    merged_df.drop(['Site Description', 'City', 'County', 'State', 'address', 'city', 'state', 'Zip', 'zip_code', 'Notes',
     'street_num'], axis=1, inplace=True)

    merged_df = merged_df[['Site #', 'order_id', 'Address', 'pickup_location']]

    return merged_df

merged_df = get_merged_and_organized_master_df()
merged_df

,Site #,order_id,Address,pickup_location
0,20,E0EB8837,5004 Wesley Street,5004 Wesley St Greenville TX 75402
1,20,78588F69,5004 Wesley Street,5004 Wesley St Greenville TX 75402
2,20,CA2588A4,5004 Wesley Street,5004 Wesley St Greenville TX 75402
3,20,ABDD1702,5004 Wesley Street,5004 Wesley St Greenville TX 75402
4,32,B6233BDE,1001 Van Alstyne Parkway,1001 W Van Alstyne Pkwy Van Alstyne TX 75495
5,53,42CEA1A6,1301 Hwy 281 North,1301 N U.S. Hwy 281 Marble Falls TX 78654
6,53,6AB38CB1,1301 Hwy 281 North,1301 N U.S. Hwy 281 Marble Falls TX 78654
7,57,B48F2460,14620 Ranch Road 12,14620 Ranch Rd 12 Wimberley TX 78676
8,57,5C857863,14620 Ranch Road 12,14620 Ranch Rd 12 Wimberley TX 78676
9,65,2ABD90DB,1402 Williams Drive,1402 Williams Dr Georgetown TX 78628


In [21]:
def store_num_to_order_ids(original_dict):
    aggregated_dict = {}
    
    for order_id, value in original_dict.items():
        if value not in aggregated_dict:
            aggregated_dict[value] = set()
        aggregated_dict[value].add(order_id)
    return aggregated_dict


In [12]:
def get_store_num_to_order_ids():
    order_id_to_store_num = merged_df.set_index('order_id')['Site #'].to_dict()
    store_num_to_order_ids_map = store_num_to_order_ids(order_id_to_store_num)
    return store_num_to_order_ids_map
store_num_to_order_ids_map = get_store_num_to_order_ids()
store_num_to_order_ids_map

{'6': {'0ED5F1C8', '8C4F6250'},
 '10': {'7DE4AD15'},
 '16': {'AF6DBE37'},
 '20': {'D7BD216C'},
 '53': {'C032FE0A'},
 '57': {'8BE8532F', 'D84A9C02'},
 '60': {'60BF65AB'},
 '68': {'3286DC34'},
 '70': {'92022795', 'F9A3C1CE'},
 '80': {'234FE808', '91EC3C1E'},
 '89': {'4761F061'}}

In [13]:
def add_store_numbers_to_orders(orders, store_num_to_order_ids_map):
    for order in orders:
        order_id = order.get('Order')
        found = False  # Variable to keep track of whether the order_id was found

        for store_num, order_id_set in store_num_to_order_ids_map.items():
            if order_id in order_id_set:
                order['Store Number'] = store_num
                found = True
                break  # Exit the loop once the order_id is found

        if not found:
            order['Store Number'] = 'N/A'
    
    return orders

add_store_numbers_to_orders(orders, store_num_to_order_ids_map)

[{'Order': '92022795',
  'Delivered': 'The order was delivered at 11:45 PM on October 2, 2023.',
  'Pick Up Location': '800 W Interstate 20, Big Spring, TX 79720, USA',
  'Order Details': {'1 × Barcel Takis Fuego Hot Chili Pepper & Lime Corn Snacks (4 oz) (Chips & Popcorn TX)': '$4.19',
   ' 1 × Hostess Powdered Donettes Bag (10.5oz) (Donuts TX)': '$6.39',
   ' 1 × Big Red Bottle (20oz) (Soda TX)': '$3.79',
   ' 1 × Cheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)': '$3.19'},
  'Subtotal': '$17.56',
  'Subtotal Tax': '$0.31',
  'Commission (23%)': '-$4.04',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$13.83',
  'Transaction #8120214500 - Delivery': '$13.83',
  'Store Number': '70'},
 {'Order': '3286DC34',
  'Delivered': 'The order was delivered at 11:45 PM on October 2, 2023.',
  'Pick Up Location': '145 Lehman Rd, Kyle, TX 78640, USA',
  'Order Details': {"1 × Ben & Jerry's Chunky Monkey Ice Cream (16 oz) (Ice Cream TX)": '$12.49'},
  'Subtotal': '$12.49',
  'Sub

In [61]:
master_df

,Site #,Site Description,Address,City,County,State,Zip,Notes,street_num
Store #,,,,,,,,,
1,1,Kwik Chek #1,2226 North Center,Bonham,Fannin,TX,75418,NaN,2226
3,3,Kwik Chek #3,971 FM 273,Bonham,Fannin,TX,75418,NaN,971
5,5,Kwik Chek #5,NaN,Bonham,Fannin,TX,75418,Not Open Yet,nan
6,6,Kwik Chek #6,816 North Center,Bonham,Fannin,TX,75418,NaN,816
7,7,Kwik Chek #7,101 Hwy 69,Celeste,Hunt,TX,75423,NaN,101
8,8,Kwik Chek #8,308 Hwy 69,Leonard,Fannin,TX,75452,NaN,308
10,10,Kwik Chek #10,1741 North Hwy 121,Bonham,Fannin,TX,75418,NaN,1741
12,12,Kwik Chek #12,102 S Hwy 78,Achille,Bryan,OK,74720,NaN,102
13,13,Kwik Chek #13,102 West Main,Calera,Bryan,OK,74730,NaN,102


In [62]:
unique_cities = set(master_df['City'].dropna().unique())
unique_cities

{'Achille',
 'Antlers',
 'Asherton',
 'Austin',
 'Bee Cave',
 'Big Spring',
 'Boerne',
 'Bonham',
 'Bryan',
 'Buchanan Dam',
 'Calera',
 'Carrizo Springs',
 'Celeste',
 'Cottonwood Shores',
 'Crystal City',
 'Durant',
 'Eagle Pass',
 'Fredericksburg',
 'Georgetown',
 'Greenville',
 'Helotes',
 'Honey Grove',
 'Hugo',
 'Johnson City',
 'Kiowa',
 'Kyle',
 'La Pryor',
 'Lakeway',
 'Laredo',
 'Leander',
 'Leonard',
 'Marble Falls',
 'Midland',
 'Paris',
 'Spicewood',
 'Temple',
 'Three Rivers',
 'Trenton',
 'Van Alstyne',
 'Wimberley',
 'Wolfe City',
 'Wylie'}

In [68]:
order_to_location_df

,order_id,pickup_location
0,2ABD90DB,1402 Williams Dr Georgetown TX 78628
1,42CEA1A6,1301 N U.S. Hwy 281 Marble Falls TX 78654
2,6AB38CB1,1301 N U.S. Hwy 281 Marble Falls TX 78654
3,4379B60E,7045 W Del Mar Blvd Laredo TX 78041
4,0A150B93,2503 TX-349 Midland TX 79706
5,E0EB8837,5004 Wesley St Greenville TX 75402
6,77896463,7045 W Del Mar Blvd Laredo TX 78041
7,B48F2460,14620 Ranch Rd 12 Wimberley TX 78676
8,78588F69,5004 Wesley St Greenville TX 75402
9,5C857863,14620 Ranch Rd 12 Wimberley TX 78676


In [67]:
order_to_location_df['pickup_location'] = order_to_location_df['pickup_location'].str.replace(',', '')
order_to_location_df['pickup_location'] = order_to_location_df['pickup_location'].str.replace(' USA', '')
order_to_location_df

,order_id,pickup_location
0,2ABD90DB,1402 Williams Dr Georgetown TX 78628
1,42CEA1A6,1301 N U.S. Hwy 281 Marble Falls TX 78654
2,6AB38CB1,1301 N U.S. Hwy 281 Marble Falls TX 78654
3,4379B60E,7045 W Del Mar Blvd Laredo TX 78041
4,0A150B93,2503 TX-349 Midland TX 79706
5,E0EB8837,5004 Wesley St Greenville TX 75402
6,77896463,7045 W Del Mar Blvd Laredo TX 78041
7,B48F2460,14620 Ranch Rd 12 Wimberley TX 78676
8,78588F69,5004 Wesley St Greenville TX 75402
9,5C857863,14620 Ranch Rd 12 Wimberley TX 78676
